In [1]:
# =======================
# 1. Install + imports
# =======================
!pip install earthengine-api geemap
!pip install tensorflow numpy pandas matplotlib scikit-learn
!pip install rasterio xarray
!pip install meteostat  # For weather data


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import ee
import geemap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import os

# Deep Learning
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Weather data
from meteostat import Point, Daily
import warnings
warnings.filterwarnings('ignore')

2025-11-25 12:12:58.325648: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-25 12:12:58.325775: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-25 12:12:58.333034: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-25 12:12:58.829558: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Authenticate and initialize Earth Engine
# First time: This will open a browser for authentication
try:
    ee.Initialize(project='pelagic-garage-478911-f9')
    print("Earth Engine initialized successfully!")
except:
    ee.Authenticate()
    ee.Initialize(project='pelagic-garage-478911-f9')
    print("Earth Engine authenticated and initialized!")

Earth Engine initialized successfully!


In [4]:
# Hartbeespoort Dam coordinates (South Africa)
HARTBEESPOORT_CENTER = [-25.73, 27.87]  # lat, lon
HARTBEESPOORT_COORDS = [
    [27.7800, -25.7200],  # Northwest
    [27.9100, -25.7200],  # Northeast
    [27.9100, -25.7900],  # Southeast
    [27.7800, -25.7900]  # Southwest
    # [27.8200, -25.7100]   # Close polygon
]

# Create Area of Interest (AOI)
aoi = ee.Geometry.Polygon(HARTBEESPOORT_COORDS)

print("Area of Interest (Hartbeespoort Dam) defined successfully!")

Area of Interest (Hartbeespoort Dam) defined successfully!


In [5]:
def create_water_mask(aoi):
    """
    Create a water-only mask for Hartbeespoort Dam.
    This mask excludes land areas from analysis.
    
    Uses JRC Global Surface Water dataset to identify permanent water bodies.
    """
    
    # Use JRC Global Surface Water to identify water areas
    # occurrence > 80 means water is present >80% of the time
    gsw = ee.Image('JRC/GSW1_4/GlobalSurfaceWater')
    water_occurrence = gsw.select('occurrence')
    
    # Create water mask: areas with >50% water occurrence
    # This excludes land and only keeps the dam water surface
    water_mask = water_occurrence.gt(50)
    
    # Clip to AOI
    water_mask = water_mask.clip(aoi)
    
    return water_mask


def apply_water_mask(image, water_mask):
    """
    Apply water mask to image so spectral indices only show on water.
    This prevents MNDWI/NDVI from showing on land areas.
    """
    return image.updateMask(water_mask)


# Create the water mask for our AOI
water_mask = create_water_mask(aoi)
print("Water mask created - this will exclude land areas from analysis!")


Water mask created - this will exclude land areas from analysis!


In [6]:
def detect_water_hyacinth_masked(image, water_mask):
    """
    Detect water hyacinth with water masking applied.
    This ensures indices only appear on water, not land.
    """
    # First apply water mask to image
    image_masked = apply_water_mask(image, water_mask)
    
    # Calculate indices on masked image
    image_with_indices = calculate_water_hyacinth_indices(image_masked)
    
    # Create water hyacinth mask based on spectral characteristics
    water_hyacinth_mask = (
        # image_with_indices.select('NDVI').gt(0.3)          # Vegetation present
        # .And(image_with_indices.select('NDVI').lt(0.85))   # Not dense forest
        # .And(image_with_indices.select('MNDWI').lt(0.1))   # Not open water
        # .And(image_with_indices.select('NDWI').gt(-0.3))   # Has moisture
        # .And(image_with_indices.select('NDWI').lt(0.3))    # But not pure water
        image_with_indices.select('NDWI').lt(0.3)    # But not pure water
        # .And(image_with_indices.select('EVI').gt(0.2))     # Active photosynthesis
    )
    
    # Add mask as a band
    return image_with_indices.addBands(
        water_hyacinth_mask.rename('WATER_HYACINTH')
    )


def get_sentinel2_time_series_masked(start_date, end_date, aoi, water_mask, cloud_threshold=20):
    """
    Fetch Sentinel-2 imagery time series with water masking applied.
    """
    
    collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                  .filterBounds(aoi)
                  .filterDate(start_date, end_date)
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_threshold))
                  .sort('system:time_start'))
    
    # Apply water hyacinth detection to each image with water mask
    collection_with_detection = collection.map(
        lambda img: detect_water_hyacinth_masked(img, water_mask)
    )
    
    return collection_with_detection


print("✓ Water masking functions defined!")
print("  - Spectral indices will only show on water, not land")
print("  - Ready for neural network input")


✓ Water masking functions defined!
  - Spectral indices will only show on water, not land
  - Ready for neural network input


In [7]:
def calculate_water_hyacinth_indices(image):
    """
    Calculate spectral indices to detect water hyacinth.
    
    Water hyacinth has UNIQUE spectral characteristics:
    - HIGH NIR reflectance (like vegetation)
    - Floating on water (MNDWI helps separate from open water)
    - High water content (NDWI detects this)
    - Active photosynthesis (NDVI shows this)
    
    Key bands in Sentinel-2:
    - B2: Blue (490nm)
    - B3: Green (560nm)
    - B4: Red (665nm)
    - B5: Red Edge 1 (705nm)
    - B6: Red Edge 2 (740nm)
    - B8: NIR (842nm)
    - B11: SWIR1 (1610nm)
    - B12: SWIR2 (2190nm)
    """
    
    # NDVI = (NIR - Red) / (NIR + Red)
    # High NDVI (>0.3) indicates vegetation
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    
    # NDWI (McFeeters) = (Green - NIR) / (Green + NIR)
    # Water hyacinth has NEGATIVE NDWI (unlike open water which is positive)
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    
    # MNDWI = (Green - SWIR) / (Green + SWIR)
    # Helps distinguish water hyacinth from open water
    mndwi = image.normalizedDifference(['B3', 'B11']).rename('MNDWI')
    
    # EVI = 2.5 * ((NIR - Red) / (NIR + 6*Red - 7.5*Blue + 1))
    # Enhanced vegetation index - more sensitive than NDVI
    evi = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
        {
            'NIR': image.select('B8'),
            'RED': image.select('B4'),
            'BLUE': image.select('B2')
        }
    ).rename('EVI')
    
    # Red Edge NDVI = (NIR - RedEdge) / (NIR + RedEdge)
    # Red edge is very sensitive to vegetation health
    red_edge_ndvi = image.normalizedDifference(['B8', 'B6']).rename('RED_EDGE_NDVI')
    
    # SAVI = ((NIR - Red) / (NIR + Red + L)) * (1 + L)
    # Soil-Adjusted Vegetation Index (L=0.5 for moderate vegetation)
    savi = image.expression(
        '((NIR - RED) / (NIR + RED + 0.5)) * 1.5',
        {
            'NIR': image.select('B8'),
            'RED': image.select('B4')
        }
    ).rename('SAVI')
    
    return image.addBands([ndvi, ndwi, mndwi, evi, red_edge_ndvi, savi])


def calculate_coverage_percentage(image, aoi):
    """
    Calculate water hyacinth coverage percentage for the AOI.
    """
    
    # Get the water hyacinth mask
    wh_mask = image.select('WATER_HYACINTH')
    
    # Calculate statistics
    stats = wh_mask.reduceRegion(
        reducer=ee.Reducer.sum().combine(
            reducer2=ee.Reducer.count(),
            sharedInputs=True
        ),
        geometry=aoi,
        scale=10,  # 10m resolution
        maxPixels=1e9
    )
    
    return stats


In [8]:
def extract_time_series_data(collection, aoi):
    """
    Extract water hyacinth coverage over time.
    Returns a pandas DataFrame with dates and coverage percentages.
    """
    
    dates = []
    coverage_percentages = []
    
    # Get list of images
    image_list = collection.toList(collection.size())
    size = collection.size().getInfo()
    
    print(f"Processing {size} images...")
    
    for i in range(size):
        image = ee.Image(image_list.get(i))
        
        # Get date
        date = datetime.fromtimestamp(
            image.get('system:time_start').getInfo() / 1000
        )
        
        # Calculate coverage
        stats = calculate_coverage_percentage(image, aoi).getInfo()
        
        if 'WATER_HYACINTH_sum' in stats and 'WATER_HYACINTH_count' in stats:
            wh_pixels = stats['WATER_HYACINTH_sum']
            total_pixels = stats['WATER_HYACINTH_count']
            
            if total_pixels > 0:
                coverage_pct = (wh_pixels / total_pixels) * 100
                dates.append(date)
                coverage_percentages.append(coverage_pct)
                
                print(f"{date.strftime('%Y-%m-%d')}: {coverage_pct:.2f}% coverage")
    
    df = pd.DataFrame({
        'date': dates,
        'coverage_percentage': coverage_percentages
    })
    
    return df


print("✓ Time series extraction functions defined!")
print("  Use with get_sentinel2_time_series_masked() from Part 6")


✓ Time series extraction functions defined!
  Use with get_sentinel2_time_series_masked() from Part 6


In [9]:
def visualize_water_hyacinth_detection(collection, aoi, water_mask):
    """
    Create an interactive map showing water hyacinth detection.
    All indices are properly masked to show only on water.
    """
    
    Map = geemap.Map()
    center = aoi.centroid().coordinates().getInfo()[::-1]
    Map.setCenter(center[1], center[0], 13)
    
    # Get most recent image
    latest_image = collection.sort('system:time_start', False).first()
    
    # RGB visualization (base layer - full image)
    rgb_vis = {
        'bands': ['B4', 'B3', 'B2'],
        'min': 0,
        'max': 3000,
        'gamma': 1.4
    }
    
    # Water hyacinth detection visualization
    wh_vis = {
        'bands': ['WATER_HYACINTH'],
        'min': 0,
        'max': 1,
        'palette': ['white', 'darkgreen']  # White=no hyacinth, Dark green=hyacinth
    }
    
    # NDVI visualization (masked to water only)
    ndvi_vis = {
        'bands': ['NDVI'],
        'min': -0.2,
        'max': 0.8,
        'palette': ['blue', 'white', 'green']
    }
    
    # MNDWI visualization (masked to water only)
    mndwi_vis = {
        'bands': ['MNDWI'],
        'min': -0.5,
        'max': 0.5,
        'palette': ['brown', 'white', 'cyan']
    }
    
    # NDWI visualization (masked to water only)
    ndwi_vis = {
        'bands': ['NDWI'],
        'min': -0.3,
        'max': 0.3,
        'palette': ['red', 'white', 'blue']
    }
    
    # Water mask visualization
    water_vis = {
        'palette': ['white', 'lightblue']
    }
    
    # Add layers to map
    Map.addLayer(latest_image, rgb_vis, 'RGB (Satellite Image)')
    Map.addLayer(water_mask, water_vis, 'Water Body Mask', False)
    Map.addLayer(latest_image, ndvi_vis, 'NDVI (Vegetation - Water Only)', False)
    Map.addLayer(latest_image, mndwi_vis, 'MNDWI (Water Index - Water Only)', False)
    Map.addLayer(latest_image, ndwi_vis, 'NDWI (Moisture - Water Only)', False)
    Map.addLayer(latest_image, wh_vis, 'Water Hyacinth Detection', opacity=0.7)
    Map.addLayer(aoi, {'color': 'yellow'}, 'AOI Boundary', False)
    
    print("✓ Interactive map created!")
    print("  - All indices (NDVI, MNDWI, NDWI) are masked to water body only")
    print("  - Water hyacinth shown in dark green")
    print("  - Toggle layers on/off using the layer control")
    
    return Map


print("✓ Visualization function defined!")
print("  Call with: visualize_water_hyacinth_detection(collection, aoi, water_mask)")

# visualize_water_hyacinth_detection(
#     s2_collection_masked, aoi, water_mask
# )


✓ Visualization function defined!
  Call with: visualize_water_hyacinth_detection(collection, aoi, water_mask)


In [10]:
def get_weather_data(start_date, end_date, location):
    """
    Fetch weather data from Meteostat for Hartbeespoort Dam area.
    
    Parameters:
    - start_date: Start date (datetime object)
    - end_date: End date (datetime object)
    - location: [lat, lon]
    
    Returns:
    - DataFrame with weather data
    """
    
    # Create Point for Hartbeespoort Dam
    location_point = Point(location[0], location[1])
    
    # Fetch daily weather data
    weather_data = Daily(location_point, start_date, end_date)
    weather_df = weather_data.fetch()
    
    # Reset index to make date a column
    weather_df.reset_index(inplace=True)
    weather_df.rename(columns={'time': 'date'}, inplace=True)
    
    # Select relevant columns
    weather_columns = [
        'date', 'tavg', 'tmin', 'tmax',  # Temperature
        'prcp',  # Precipitation
        'wspd',  # Wind speed
        'tsun'   # Sunshine duration
    ]
    
    available_columns = [col for col in weather_columns if col in weather_df.columns]
    weather_df = weather_df[available_columns]
    
    print(f"✓ Retrieved weather data: {len(weather_df)} days")
    print(f"  Columns: {list(weather_df.columns)}")
    
    return weather_df


# Example: Get weather data
start_dt = datetime.now() - timedelta(days=180)
end_dt = datetime.now()

weather_df = get_weather_data(start_dt, end_dt, HARTBEESPOORT_CENTER)
print("\nWeather data sample:")
print(weather_df.head())

✓ Retrieved weather data: 178 days
  Columns: ['date', 'tavg', 'tmin', 'tmax', 'prcp', 'wspd', 'tsun']

Weather data sample:
        date  tavg  tmin  tmax  prcp  wspd  tsun
0 2025-05-30  13.9   5.4  23.0   0.0  11.4  <NA>
1 2025-05-31  12.7   6.0  21.0   0.0   9.2  <NA>
2 2025-06-01  12.4   5.0  21.0   0.0   7.4  <NA>
3 2025-06-02  11.6   2.0  22.0   0.0   6.1  <NA>
4 2025-06-03  12.1   3.0  22.0   0.0   5.7  <NA>


In [11]:
def create_integrated_dataset(coverage_df, weather_df, ph_data=None):
    """
    Combine water hyacinth coverage, weather data, and pH measurements.
    
    Parameters:
    - coverage_df: DataFrame with columns ['date', 'coverage_percentage']
    - weather_df: DataFrame with weather data
    - ph_data: DataFrame with columns ['date', 'ph_value'] (optional)
    
    Returns:
    - Integrated DataFrame ready for ML
    """
    
    
    # Merge coverage with weather data
    coverage_df['date'] = pd.to_datetime(coverage_df['date'])
    weather_df['date'] = pd.to_datetime(weather_df['date'])
    
    df = pd.merge(coverage_df, weather_df, on='date', how='inner')
    
    # Add pH data if available
    if ph_data is not None:
        ph_data['date'] = pd.to_datetime(ph_data['date'])
        df = pd.merge(df, ph_data, on='date', how='left')
        # Forward fill missing pH values (pH changes slowly)
        df['ph_value'] = df['ph_value'].ffill()
    
    # Add time-based features
    df['day_of_year'] = df['date'].dt.dayofyear
    df['month'] = df['date'].dt.month
    df['week'] = df['date'].dt.isocalendar().week
    
    # Add lagged features (previous coverage)
    df['coverage_lag_1'] = df['coverage_percentage'].shift(1)
    df['coverage_lag_7'] = df['coverage_percentage'].shift(7)
    
    # Calculate rolling averages
    df['temp_7day_avg'] = df['tavg'].rolling(window=7, min_periods=1).mean()
    df['prcp_7day_sum'] = df['prcp'].rolling(window=7, min_periods=1).sum()
    
    # Drop rows with NaN from lagging
    df = df.dropna()
    
    print(f"✓ Integrated dataset created: {len(df)} samples")
    print(f"  Features: {list(df.columns)}")
    
    return df




In [12]:
def prepare_sequences(integrated_df, sequence_length=14):
    """
    Prepare input sequences and labels for LSTM model.
    Returns:
      X: (num_samples, sequence_length, num_features)
      y: (num_samples,)
      feature_cols: list of feature names
      scaler: fitted MinMaxScaler (or None if no data)
    """
    from sklearn.preprocessing import MinMaxScaler

    # Choose which columns to use as features
    feature_cols = [
        'hyacinth_percent',
        'temp',
        'precip',
        'wind_speed',
        'humidity',
        'pressure',
        'cloud_cover',
        'sunshine',
        'wind_dir_sin',
        'wind_dir_cos',
        'day_of_year'
    ]

    df = integrated_df.copy().sort_values('date')

    # Drop rows where any of the features are NaN
    df = df.dropna(subset=feature_cols)

    if df.empty:
        print("⚠ No rows left after dropping NaNs in feature columns.")
        print("  Cannot build sequences – returning empty arrays.")
        X_empty = np.empty((0, sequence_length, len(feature_cols)))
        y_empty = np.empty((0,))
        return X_empty, y_empty, feature_cols, None

    data = df[feature_cols].values

    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(data)

    X = []
    y = []

    # We predict next-day hyacinth_percent
    target = df['hyacinth_percent'].values

    for i in range(len(df) - sequence_length):
        X.append(data_scaled[i:i + sequence_length])
        y.append(target[i + sequence_length])

    X = np.array(X)
    y = np.array(y)

    if X.shape[0] == 0:
        print("⚠ Not enough rows to create any sequences with "
              f"sequence_length={sequence_length}.")
        X_empty = np.empty((0, sequence_length, len(feature_cols)))
        y_empty = np.empty((0,))
        return X_empty, y_empty, feature_cols, scaler

    print(f"Prepared sequences: X.shape={X.shape}, y.shape={y.shape}")
    return X, y, feature_cols, scaler



def build_water_hyacinth_predictor(input_shape, n_features):
    """
    Build LSTM model to predict water hyacinth growth.
    
    This model learns patterns from:
    - Historical coverage
    - Weather conditions (temp, rainfall, wind)
    - Seasonal patterns
    - pH levels (if available)
    
    It predicts future coverage percentage.
    """
    
    model = Sequential(name='WaterHyacinthPredictor')
    
    # LSTM layers to learn temporal patterns
    model.add(LSTM(128, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.3))
    
    model.add(LSTM(64, return_sequences=True))
    model.add(Dropout(0.3))
    
    model.add(LSTM(32, return_sequences=False))
    model.add(Dropout(0.3))
    
    # Dense layers for final prediction
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    
    model.add(Dense(16, activation='relu'))
    
    # Output: predicted coverage percentage (normalized 0-1)
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile
    model.compile(
        optimizer='adam',
        loss='mean_squared_error',
        metrics=['mae', 'mse']
    )
    
    return model


In [13]:
def build_cnn_lstm_predictor(input_shape):
    """
    Build CNN-LSTM hybrid model.
    
    CNN extracts local patterns from the time series.
    LSTM captures long-term temporal dependencies.
    """
    
    model = Sequential(name='CNN_LSTM_Predictor')
    
    # CNN layers for feature extraction
    model.add(Conv1D(64, kernel_size=3, activation='relu', 
                     padding='same', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.3))
    
    model.add(Conv1D(32, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.3))
    
    # LSTM layers for temporal modeling
    model.add(LSTM(64, return_sequences=True))
    model.add(Dropout(0.3))
    
    model.add(LSTM(32))
    model.add(Dropout(0.3))
    
    # Output layers
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(
        optimizer='adam',
        loss='mean_squared_error',
        metrics=['mae']
    )
    
    return model

In [14]:
def train_hyacinth_movement_model(coverage_df, weather_df, ph_data=None):
    """
    Train the complete water hyacinth movement prediction model.
    Returns:
      model, scaler, feature_cols, history
      (or all None if there is not enough data)
    """
    print("Step 1: Integrating data sources...")
    integrated_df = create_integrated_dataset(coverage_df, weather_df, ph_data)
    print(f"  Integrated dataset shape: {integrated_df.shape}")

    # Safety check: empty integrated dataset
    if integrated_df is None or integrated_df.empty:
        print("  ⚠ Integrated dataset is empty. Skipping model training.")
        return None, None, None, None

    # Safety check: minimum rows before even trying sequences
    min_rows = 30
    if len(integrated_df) < min_rows:
        print(f"  ⚠ Only {len(integrated_df)} rows (< {min_rows}). "
              "Not enough data to train LSTM. Skipping model training.")
        return None, None, None, None

    print("\nStep 2: Preparing time series sequences...")
    X, y, feature_cols, scaler = prepare_sequences(integrated_df, sequence_length=14)

    if X.shape[0] == 0:
        print("  ⚠ Sequence preparation returned 0 samples. "
              "Skipping model training.")
        return None, None, None, None

    print("\nStep 3: Splitting train/test data...")
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, shuffle=False
    )
    print(f"  Train: {X_train.shape[0]} samples, Test: {X_test.shape[0]} samples")

    print("\nStep 4: Building LSTM model...")
    model = build_water_hyacinth_predictor(
        input_shape=(X.shape[1], X.shape[2]),
        n_features=X.shape[2]
    )
    model.summary()

    print("\nStep 5: Training model...")
    early_stop = EarlyStopping(
        monitor='val_loss', patience=20, restore_best_weights=True
    )
    checkpoint = ModelCheckpoint(
        'best_wh_predictor.h5', save_best_only=True, monitor='val_loss'
    )

    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=100,
        batch_size=32,
        callbacks=[early_stop, checkpoint],
        verbose=1
    )

    print("\nStep 6: Evaluating model...")
    test_loss, test_mae, test_mse = model.evaluate(X_test, y_test, verbose=0)
    print(f"  Test Loss: {test_loss:.4f}")
    print(f"  Test MAE: {test_mae:.4f}")
    print(f"  Test MSE: {test_mse:.4f}")

    return model, scaler, feature_cols, history



# Example usage:
# model, scaler, features, history = train_hyacinth_movement_model(
#     coverage_df, weather_df, ph_data
# )

print("✓ Model training workflow defined!")


✓ Model training workflow defined!


In [15]:
# WATER HYACINTH MOVEMENT PREDICTION

def predict_hyacinth_movement(model, scaler, features, integrated_df, days_ahead=7):
    """
    Predict water hyacinth coverage for the next N days.
    
    Parameters:
    - model: Trained LSTM model
    - scaler: MinMaxScaler used during training
    - features: List of feature names
    - integrated_df: Latest integrated data
    - days_ahead: Number of days to predict into the future
    
    Returns:
    - DataFrame with predictions
    """
    
    predictions = []
    dates = []
    
    # Get the last sequence from the data
    sequence_length = 14
    last_sequence = integrated_df[features].tail(sequence_length).values
    last_sequence_scaled = scaler.transform(last_sequence)
    
    # Reshape for model input
    current_sequence = last_sequence_scaled.reshape(1, sequence_length, len(features))
    
    # Get last known date
    last_date = integrated_df['date'].iloc[-1]
    
    # Iteratively predict future values
    for day in range(days_ahead):
        # Predict next value
        next_pred_scaled = model.predict(current_sequence, verbose=0)[0, 0]
        
        # Create next feature vector (simplified - using last known weather)
        next_features_scaled = current_sequence[0, -1, :].copy()
        next_features_scaled[0] = next_pred_scaled  # Update coverage prediction
        
        # Update sequence for next prediction
        current_sequence = np.roll(current_sequence, -1, axis=1)
        current_sequence[0, -1, :] = next_features_scaled
        
        # Inverse transform to get actual coverage percentage
        # Create full feature vector for inverse transform
        full_vector = next_features_scaled.reshape(1, -1)
        coverage_pct = scaler.inverse_transform(full_vector)[0, 0]
        
        # Store prediction
        pred_date = last_date + timedelta(days=day+1)
        predictions.append(coverage_pct)
        dates.append(pred_date)
    
    # Create predictions DataFrame
    pred_df = pd.DataFrame({
        'date': dates,
        'predicted_coverage': predictions
    })
    
    return pred_df


def visualize_movement_prediction(historical_df, prediction_df):
    """
    Visualize historical coverage and predicted movement.
    """
    
    fig, ax = plt.subplots(figsize=(16, 7))
    
    # Plot historical data
    ax.plot(historical_df['date'], historical_df['coverage_percentage'],
            marker='o', linewidth=2, markersize=6, color='darkgreen',
            label='Historical Coverage')
    ax.fill_between(historical_df['date'], 0, historical_df['coverage_percentage'],
                     alpha=0.3, color='green')
    
    # Plot predictions
    ax.plot(prediction_df['date'], prediction_df['predicted_coverage'],
            marker='s', linewidth=2, markersize=8, color='red',
            linestyle='--', label='Predicted Coverage')
    ax.fill_between(prediction_df['date'], 0, prediction_df['predicted_coverage'],
                     alpha=0.2, color='red')
    
    # Add vertical line at prediction start
    ax.axvline(x=historical_df['date'].iloc[-1], color='blue',
               linestyle=':', linewidth=2, label='Prediction Start')
    
    ax.set_xlabel('Date', fontsize=13, fontweight='bold')
    ax.set_ylabel('Water Hyacinth Coverage (%)', fontsize=13, fontweight='bold')
    ax.set_title('Water Hyacinth Movement Prediction - Hartbeespoort Dam',
                 fontsize=15, fontweight='bold')
    ax.legend(loc='best', fontsize=11)
    ax.grid(True, alpha=0.3)
    
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    return fig


# Example usage:
# predictions = predict_hyacinth_movement(
#     model, scaler, features, integrated_df, days_ahead=14
# )
# fig = visualize_movement_prediction(coverage_df, predictions)

print("✓ Movement prediction functions defined!")
print("  - Predict coverage for N days ahead")
print("  - Visualize predicted movement on the dam")


✓ Movement prediction functions defined!
  - Predict coverage for N days ahead
  - Visualize predicted movement on the dam


In [16]:
# SPATIAL PREDICTION VISUALIZATION ON MAP

def visualize_predicted_movement_on_map(collection, aoi, water_mask, prediction_df):
    """
    Show predicted water hyacinth movement on an interactive map.
    """
    
    Map = geemap.Map()
    center = aoi.centroid().coordinates().getInfo()[::-1]
    Map.setCenter(center[1], center[0], 13)
    
    # Get most recent image
    latest_image = collection.sort('system:time_start', False).first()
    
    # RGB visualization
    rgb_vis = {
        'bands': ['B4', 'B3', 'B2'],
        'min': 0,
        'max': 3000,
        'gamma': 1.4
    }
    
    # Water mask visualization
    water_vis = {
        'palette': ['white', 'blue']
    }
    
    # Water hyacinth current state
    wh_vis = {
        'bands': ['WATER_HYACINTH'],
        'min': 0,
        'max': 1,
        'palette': ['white', 'darkgreen']
    }
    
    # MNDWI visualization (masked to water only)
    mndwi_vis = {
        'bands': ['MNDWI'],
        'min': -1,
        'max': 1,
        'palette': ['brown', 'white', 'cyan']
    }
    
    # Add layers
    Map.addLayer(latest_image, rgb_vis, 'RGB (Latest)')
    Map.addLayer(water_mask, water_vis, 'Water Mask (Dam Only)', False)
    Map.addLayer(latest_image, mndwi_vis, 'MNDWI (Water Only)', False)
    Map.addLayer(latest_image, wh_vis, 'Water Hyacinth (Current)', opacity=0.7)
    Map.addLayer(aoi, {'color': 'yellow'}, 'AOI Boundary')
    
    # Add prediction info as text
    if prediction_df is not None and len(prediction_df) > 0:
        avg_predicted = prediction_df['predicted_coverage'].mean()
        max_predicted = prediction_df['predicted_coverage'].max()
        
        info_text = f"""
        WATER HYACINTH MOVEMENT PREDICTION
        
        Prediction Period: Next {len(prediction_df)} days
        Average Predicted Coverage: {avg_predicted:.2f}%
        Maximum Predicted Coverage: {max_predicted:.2f}%
        
        The model predicts hyacinth movement based on:
        - Historical coverage patterns
        - Weather conditions (temp, rainfall, wind)
        - Seasonal trends
        - Water quality (pH if available)
        """
        print(info_text)
    
    return Map


# Example usage:
# map_with_predictions = visualize_predicted_movement_on_map(
#     s2_collection_masked, aoi, water_mask, predictions
# )
# map_with_predictions

print("✓ Spatial prediction visualization defined!")
print("  - Shows current water hyacinth on map")
print("  - Displays prediction statistics")
print("  - MNDWI and indices only show on water (land masked out)")


✓ Spatial prediction visualization defined!
  - Shows current water hyacinth on map
  - Displays prediction statistics
  - MNDWI and indices only show on water (land masked out)


In [17]:
def plot_coverage_time_series(df):
    """
    Plot water hyacinth coverage over time.
    """
    
    fig, ax = plt.subplots(figsize=(15, 6))
    
    ax.plot(df['date'], df['coverage_percentage'], 
            marker='o', linewidth=2, markersize=6, color='darkgreen')
    ax.fill_between(df['date'], 0, df['coverage_percentage'], 
                     alpha=0.3, color='green')
    
    ax.set_xlabel('Date', fontsize=12)
    ax.set_ylabel('Water Hyacinth Coverage (%)', fontsize=12)
    ax.set_title('Water Hyacinth Coverage at Hartbeespoort Dam', 
                 fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('water_hyacinth_coverage_timeseries.png', dpi=300)
    plt.show()


def plot_predictions(y_true, y_pred, dates):
    """
    Plot actual vs predicted coverage.
    """
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))
    
    # Time series comparison
    ax1.plot(dates, y_true, label='Actual', marker='o', linewidth=2)
    ax1.plot(dates, y_pred, label='Predicted', marker='s', 
             linewidth=2, linestyle='--')
    ax1.set_xlabel('Date', fontsize=12)
    ax1.set_ylabel('Coverage (%)', fontsize=12)
    ax1.set_title('Actual vs Predicted Water Hyacinth Coverage', 
                  fontsize=14, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Scatter plot
    ax2.scatter(y_true, y_pred, alpha=0.6, s=100)
    ax2.plot([0, 100], [0, 100], 'r--', linewidth=2, label='Perfect Prediction')
    ax2.set_xlabel('Actual Coverage (%)', fontsize=12)
    ax2.set_ylabel('Predicted Coverage (%)', fontsize=12)
    ax2.set_title('Prediction Accuracy', fontsize=14, fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('prediction_results.png', dpi=300)
    plt.show()


def plot_feature_importance_over_time(df):
    """
    Visualize how different factors correlate with coverage.
    """
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    
    # Temperature vs Coverage
    axes[0, 0].scatter(df['tavg'], df['coverage_percentage'], 
                       alpha=0.6, c=df['day_of_year'], cmap='viridis')
    axes[0, 0].set_xlabel('Average Temperature (°C)')
    axes[0, 0].set_ylabel('Coverage (%)')
    axes[0, 0].set_title('Temperature vs Coverage')
    
    # Precipitation vs Coverage
    axes[0, 1].scatter(df['prcp'], df['coverage_percentage'], 
                       alpha=0.6, c=df['day_of_year'], cmap='viridis')
    axes[0, 1].set_xlabel('Precipitation (mm)')
    axes[0, 1].set_ylabel('Coverage (%)')
    axes[0, 1].set_title('Precipitation vs Coverage')
    
    # pH vs Coverage (if available)
    if 'ph_value' in df.columns:
        axes[1, 0].scatter(df['ph_value'], df['coverage_percentage'], 
                           alpha=0.6, c=df['day_of_year'], cmap='viridis')
        axes[1, 0].set_xlabel('pH Value')
        axes[1, 0].set_ylabel('Coverage (%)')
        axes[1, 0].set_title('pH vs Coverage')
    
    # Seasonal pattern
    monthly_avg = df.groupby('month')['coverage_percentage'].mean()
    axes[1, 1].bar(monthly_avg.index, monthly_avg.values, color='green', alpha=0.7)
    axes[1, 1].set_xlabel('Month')
    axes[1, 1].set_ylabel('Average Coverage (%)')
    axes[1, 1].set_title('Seasonal Pattern')
    axes[1, 1].set_xticks(range(1, 13))
    
    plt.tight_layout()
    plt.savefig('feature_analysis.png', dpi=300)
    plt.show()

In [18]:
def predict_on_sentinel2_image(model, image_path, output_path):
    """
    Run inference on a full Sentinel-2 image and generate coverage map
    """
    
    import rasterio
    from rasterio.windows import Window
    
    with rasterio.open(image_path) as src:
        # Get image dimensions
        width = src.width
        height = src.height
        
        # Create output array
        prediction_map = np.zeros((height, width), dtype=np.uint8)
        
        tile_size = 128
        
        # Slide window across image
        for i in range(0, height - tile_size, tile_size):
            for j in range(0, width - tile_size, tile_size):
                # Read tile
                window = Window(j, i, tile_size, tile_size)
                tile = src.read([1, 2, 3], window=window)
                tile = np.transpose(tile, (1, 2, 0))
                
                # Normalize
                tile = tile / tile.max()
                tile = np.expand_dims(tile, axis=0)
                
                # Predict
                pred = model.predict(tile, verbose=0)
                pred_class = np.argmax(pred)
                
                # Store prediction
                prediction_map[i:i+tile_size, j:j+tile_size] = pred_class
        
        # Save prediction map
        profile = src.profile
        profile.update(dtype=rasterio.uint8, count=1)
        
        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(prediction_map, 1)
    
    print(f"Prediction map saved to {output_path}")
    
    return prediction_map


def calculate_coverage_statistics(prediction_map, class_names):
    """
    Calculate coverage statistics from prediction map
    """
    
    total_pixels = prediction_map.size
    
    print("\nCoverage Statistics:")
    print("-" * 50)
    
    for class_idx, class_name in enumerate(class_names):
        count = np.sum(prediction_map == class_idx)
        percentage = (count / total_pixels) * 100
        print(f"{class_name}: {count} pixels ({percentage:.2f}%)")
    
    # Calculate total water hyacinth coverage
    water_hyacinth_pixels = np.sum(prediction_map == 0)
    coverage_percentage = (water_hyacinth_pixels / total_pixels) * 100
    
    print(f"\nTotal Water Hyacinth Coverage: {coverage_percentage:.2f}%")
    
    return coverage_percentage


In [19]:
def save_model_and_metadata(model, class_names, save_path='models/'):
    """
    Save trained model and associated metadata
    """
    
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    # Save model
    model.save(os.path.join(save_path, 'water_hyacinth_cnn.h5'))
    
    # Save metadata
    metadata = {
        'class_names': class_names,
        'input_shape': model.input_shape[1:],
        'n_classes': len(class_names),
        'created_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    
    with open(os.path.join(save_path, 'metadata.json'), 'w') as f:
        json.dump(metadata, f, indent=4)
    
    print(f"Model and metadata saved to {save_path}")


def load_model_and_metadata(model_path='models/'):
    """
    Load trained model and metadata
    """
    
    model = keras.models.load_model(os.path.join(model_path, 'water_hyacinth_cnn.h5'))
    
    with open(os.path.join(model_path, 'metadata.json'), 'r') as f:
        metadata = json.load(f)
    
    print("Model loaded successfully!")
    print(f"Classes: {metadata['class_names']}")
    
    return model, metadata


In [20]:
# ============================================================================
# SIMPLE USAGE EXAMPLE - Run this cell after running all cells above
# ============================================================================

# Step 1: Create water mask
print("Creating water mask to exclude land areas...")
water_mask = create_water_mask(aoi)
print("✓ Water mask created!")

# Step 2: Fetch Sentinel-2 imagery with water masking
print("\nFetching Sentinel-2 imagery (last 6 months)...")
end_date = datetime.now().strftime('%Y-%m-%d')
start_date = (datetime.now() - timedelta(days=180)).strftime('%Y-%m-%d')

s2_collection_masked = get_sentinel2_time_series_masked(
    start_date, end_date, aoi, water_mask, cloud_threshold=20
)
n_images = s2_collection_masked.size().getInfo()
print(f"✓ Found {n_images} cloud-free images")

# Step 3: Visualize the results
print("\nCreating interactive map...")
interactive_map = visualize_water_hyacinth_detection(
    s2_collection_masked, aoi, water_mask
)
print("✓ Map ready! Display it below:")

# Display the map
interactive_map

# coverage_df = extract_time_series_data(s2_collection_masked, aoi)
# print(coverage_df.head())


# # Example pH data (you would provide your actual measurements)
# ph_example = pd.DataFrame({
#     'date': pd.date_range(start=start_dt, end=end_dt, freq='7D'),
#     'ph_value': np.random.uniform(7.0, 8.5, size=len(pd.date_range(start=start_dt, end=end_dt, freq='7D')))
# })

# # Uncomment when you have actual coverage data:
# integrated_df = create_integrated_dataset(coverage_df, weather_df, ph_example)
# integrated_df.to_csv('integrated_dataset.csv', index=False)

Creating water mask to exclude land areas...
✓ Water mask created!

Fetching Sentinel-2 imagery (last 6 months)...
✓ Found 62 cloud-free images

Creating interactive map...
✓ Interactive map created!
  - All indices (NDVI, MNDWI, NDWI) are masked to water body only
  - Water hyacinth shown in dark green
  - Toggle layers on/off using the layer control
✓ Map ready! Display it below:


Map(center=[-25.755010991102683, 27.845000000001107], controls=(WidgetControl(options=['position', 'transparen…

In [ ]:
# COMPLETE END-TO-END WORKFLOW
# Run this cell to execute the entire pipeline

def run_complete_workflow(start_date, end_date, aoi, days_to_predict=14):
    """
    Complete workflow for water hyacinth tracking and prediction.
    
    Steps:
    1. Create water mask (exclude land)
    2. Fetch Sentinel-2 data with masking
    3. Extract coverage time series
    4. Get weather data
    5. Integrate datasets
    6. Train prediction model
    7. Predict future movement
    8. Visualize results
    """
    
    print("="*70)
    print("WATER HYACINTH TRACKING AND PREDICTION WORKFLOW")
    print("Hartbeespoort Dam, South Africa")
    print("="*70)
    
    # Step 1: Create water mask
    print("\n[1/8] Creating water mask...")
    water_mask = create_water_mask(aoi)
    print("      ✓ Water mask created - land areas will be excluded")
    
    # Step 2: Fetch Sentinel-2 data with water masking
    print("\n[2/8] Fetching Sentinel-2 imagery...")
    s2_collection = get_sentinel2_time_series_masked(
        start_date, end_date, aoi, water_mask
    )
    n_images = s2_collection.size().getInfo()
    print(f"      ✓ Found {n_images} images")
    
    # Step 3: Extract coverage time series
    print("\n[3/8] Extracting water hyacinth coverage time series...")
    coverage_df = extract_time_series_data(s2_collection, aoi)
    print(f"      ✓ Extracted {len(coverage_df)} data points")
    coverage_df.to_csv('hartbeespoort_coverage.csv', index=False)
    
    # Step 4: Get weather data
    print("\n[4/8] Fetching weather data...")
    start_dt = datetime.strptime(start_date, '%Y-%m-%d')
    end_dt = datetime.strptime(end_date, '%Y-%m-%d')
    weather_df = get_weather_data(start_dt, end_dt, HARTBEESPOORT_CENTER)
    print(f"      ✓ Retrieved {len(weather_df)} weather records")
    
    # Step 5: Integrate datasets
    print("\n[5/8] Integrating coverage and weather data...")
    integrated_df = create_integrated_dataset(coverage_df, weather_df)
    print(f"      ✓ Integrated dataset: {integrated_df.shape}")
    integrated_df.to_csv('hartbeespoort_integrated.csv', index=False)
    
    # Step 6: Train prediction model
    print("\n[6/8] Training LSTM prediction model...")
    model, scaler, features, history = train_hyacinth_movement_model(
        coverage_df, weather_df
    )

    if model is None:
        print("      ⚠ Model was not trained (insufficient data).")
        print("        Skipping prediction and LSTM-based visualizations.")
        predictions = None
    else:
        print("      ✓ Model trained successfully")

        print(f"\n[7/8] Predicting movement for next {days_to_predict} days...")
        predictions = predict_hyacinth_movement(
            model, scaler, features, integrated_df, days_ahead=days_to_predict
        )
        print(f"      ✓ Predictions generated")
        predictions.to_csv('hartbeespoort_predictions.csv', index=False)

    
    # Step 8: Visualize results
    print("\n[8/8] Creating visualizations...")

    if predictions is not None:
        fig = visualize_movement_prediction(coverage_df, predictions)
        fig.savefig('hartbeespoort_prediction.png', dpi=300, bbox_inches='tight')

        map_display = visualize_predicted_movement_on_map(
            s2_collection, aoi, water_mask, predictions
        )
        print("      ✓ Visualizations created (with predictions)")
    else:
        map_display = visualize_water_hyacinth_detection(
            s2_collection, aoi, water_mask
        )
        print("      ✓ Basic map created (no prediction available)")

    
    print("\n" + "="*70)
    print("WORKFLOW COMPLETE!")
    print("="*70)
    print("\nOutputs:")
    print("  - hartbeespoort_coverage.csv: Historical coverage data")
    print("  - hartbeespoort_integrated.csv: Integrated dataset")
    print("  - hartbeespoort_predictions.csv: Predictions")
    print("  - hartbeespoort_prediction.png: Visualization")
    print("  - best_wh_predictor.h5: Trained model")
    
    return {
        'coverage_df': coverage_df,
        'integrated_df': integrated_df,
        'predictions': predictions,
        'model': model,
        'scaler': scaler,
        'features': features,
        'map': map_display
    }


# To run the complete workflow, uncomment the following:
results = run_complete_workflow(
    start_date='2017-01-01',
    end_date='2025-11-20',
    aoi=aoi,
    days_to_predict=14
)
# 
# Display the map
results['map']

print("✓ Complete workflow defined!")
print("  Uncomment the code above to run the entire pipeline.")


WATER HYACINTH TRACKING AND PREDICTION WORKFLOW
Hartbeespoort Dam, South Africa

[1/8] Creating water mask...
      ✓ Water mask created - land areas will be excluded

[2/8] Fetching Sentinel-2 imagery...
      ✓ Found 658 images

[3/8] Extracting water hyacinth coverage time series...
Processing 658 images...
2017-03-16: 21.35% coverage
2017-04-05: 31.50% coverage
2017-04-20: 28.40% coverage
2017-05-17: 37.50% coverage
2017-05-22: 28.61% coverage
2017-06-01: 40.52% coverage
2017-06-06: 50.57% coverage
2017-06-14: 49.04% coverage
2017-07-11: 61.40% coverage
2017-08-10: 81.26% coverage
2017-09-04: 49.00% coverage
2017-09-09: 67.74% coverage
2017-09-22: 97.00% coverage
2017-10-04: 64.66% coverage
2017-10-19: 51.79% coverage
2017-11-03: 62.36% coverage
2017-11-18: 37.78% coverage
2017-12-28: 23.67% coverage
2018-01-02: 33.15% coverage
2018-04-07: 27.93% coverage
2018-06-11: 29.31% coverage
2018-06-19: 41.74% coverage
2018-06-24: 29.89% coverage
2018-08-30: 32.93% coverage
2018-09-19: 41.8